In [22]:
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer, get_scheduler
import torch
from torch.utils.data import DataLoader, Dataset

from peft import get_peft_model, LoraConfig
from sentence_transformers import SentenceTransformer, util

from nltk.translate.bleu_score import sentence_bleu

! pip install rouge-score nltk
from rouge_score import rouge_scorer

import numpy as np

from itertools import product

In [23]:
# import os
# os.chdir('C:/Users/reese/OneDrive/Documents/MIDS/Fall 2024/DATASCI 266/Final/Data')

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [24]:
# Load the JSON data
with open('/content/drive/My Drive/w266 Final Project/data/Adele_cleaned_songs.json', 'r') as file: #Choose between Johhny Cash, Adele, Elvis
    songs = json.load(file)

# Combine all lyrics into a single string
lyrics_data = "\n\n".join([song['lyrics'] for song in songs if song['lyrics'] is not None])
print(f"Total length of lyrics data: {len(lyrics_data)} characters")

# Split the lyrics data into smaller chunks
chunk_size = 1024  # Keep chunk size within 1024 tokens
lyrics_chunks = [lyrics_data[i:i + chunk_size] for i in range(0, len(lyrics_data), chunk_size)]

Total length of lyrics data: 88122 characters


In [25]:
# Load pre-trained model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [26]:
# Configure LoRA adaptation
config = LoraConfig(r = 8, lora_alpha = 32, lora_dropout = 0.1)
model = get_peft_model(model, config)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [27]:
# Tokenize each chunk
tokenized_inputs = []
for chunk in lyrics_chunks:
    inputs = tokenizer(chunk, return_tensors='pt', max_length=1024, truncation=True)
    tokenized_inputs.append(inputs)

print(f"Number of chunks processed: {len(tokenized_inputs)}")

Number of chunks processed: 87


In [28]:
# Prepare Dataset and DataLoader
class LyricsDataset(Dataset):
    def __init__(self, input_ids):
        self.input_ids = input_ids

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.input_ids[idx]

# Flatten tokenized inputs for Dataset
flattened_input_ids = [item['input_ids'].squeeze() for item in tokenized_inputs]
dataset = LyricsDataset(flattened_input_ids)
dataloader = DataLoader(dataset, batch_size = 1, shuffle = True)

In [29]:
# Fine-Tune the GPT-2 Model with LoRA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

#gradient_accumulation_steps = 4  # Combine gradients over multiple steps
optimizer = torch.optim.AdamW(model.parameters(), lr = 2e-5, weight_decay = 0.01)

epoch = 6

#num_training_steps = len(dataloader) * epoch
#scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

for e in range(epoch):  # Example: 3 epochs
    for batch in dataloader:
        input_ids, labels = batch
        input_ids, labels = input_ids.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        print(f"Epoch {e+1} Loss: {loss.item()}")

Epoch 1 Loss: 3.550879955291748
Epoch 1 Loss: 3.0761783123016357
Epoch 1 Loss: 2.76218581199646
Epoch 1 Loss: 2.1575231552124023
Epoch 1 Loss: 3.2104814052581787
Epoch 1 Loss: 3.4289679527282715
Epoch 1 Loss: 2.6670243740081787
Epoch 1 Loss: 3.492044448852539
Epoch 1 Loss: 2.595848321914673
Epoch 1 Loss: 3.0029027462005615
Epoch 1 Loss: 2.6925830841064453
Epoch 1 Loss: 2.4923760890960693
Epoch 1 Loss: 3.009255886077881
Epoch 1 Loss: 2.0820345878601074
Epoch 1 Loss: 2.853703737258911
Epoch 1 Loss: 2.714304208755493
Epoch 1 Loss: 2.3741254806518555
Epoch 1 Loss: 2.1432321071624756
Epoch 1 Loss: 3.045966148376465
Epoch 1 Loss: 3.121664524078369
Epoch 1 Loss: 2.5353775024414062
Epoch 1 Loss: 3.189361095428467
Epoch 1 Loss: 2.3722357749938965
Epoch 1 Loss: 2.967219829559326
Epoch 1 Loss: 2.307900905609131
Epoch 1 Loss: 3.076984167098999
Epoch 1 Loss: 2.6093368530273438
Epoch 1 Loss: 4.890334606170654
Epoch 1 Loss: 2.8869788646698
Epoch 1 Loss: 2.547220468521118
Epoch 1 Loss: 3.5669026374816

In [30]:
# Generate New Lyrics with the LoRA-adapted Model
model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2SdpaAttention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (l

In [31]:
# Load the pre-trained Sentence-BERT model
STS_model = SentenceTransformer('all-MiniLM-L6-v2')

# Semantic similarity calculation
def compute_semantic_similarity(generated_lyrics, reference_lyrics):
    # Encode the sentences into embeddings
    generated_embedding = STS_model.encode(generated_lyrics, convert_to_tensor=True)
    reference_embedding = STS_model.encode(reference_lyrics, convert_to_tensor=True)

    # Compute cosine similarity between the embeddings
    similarity_score = util.pytorch_cos_sim(generated_embedding, reference_embedding)
    return similarity_score.item()

In [32]:
# BLEU score calculation
def compute_bleu_score(generated_lyrics, reference_lyrics):
    reference_tokens = reference_lyrics.split()  # Tokenize the reference lyrics
    generated_tokens = generated_lyrics.split()  # Tokenize the generated lyrics
    bleu_score = sentence_bleu([reference_tokens], generated_tokens)  # Compute BLEU score
    return bleu_score

In [33]:
# ROUGE score calculation
def compute_rouge_score(generated_lyrics, reference_lyrics):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_lyrics, generated_lyrics)
    return scores

In [34]:
# Define parameter grid
param_grid = {
    "top_k": [10, 20, 30],
    "top_p": [0.6, 0.7, 0.8, 0.9],
    "temperature": [0.6, 0.7, 0.9],
}

# Generate all combinations of parameters
param_combinations = list(product(
    param_grid["top_k"],
    param_grid["top_p"],
    param_grid["temperature"],
))

In [35]:
# Initialize variables
best_params = None
best_score = -float("inf")
results = []

prompts = [song['lyrics'].split('\n')[1] for song in songs if song['lyrics'] is not None]
reference_lyrics = [song['lyrics'] for song in songs if song['lyrics'] is not None]

# # Ensure pad_token is properly defined
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add custom pad token if not present
#     model.resize_token_embeddings(len(tokenizer))  # Resize model embeddings to include new token

# # Set `pad_token_id` to the integer ID of the pad token
# tokenizer.pad_token_id = tokenizer.eos_token_id if tokenizer.pad_token_id is None else tokenizer.pad_token_id
# model.config.pad_token_id = tokenizer.pad_token_id  # Ensure consistency

# Iterate through parameter combinations
for top_k, top_p, temperature in param_combinations:
    print(f"Testing parameters: top_k={top_k}, top_p={top_p}, temperature={temperature}")

    semantic_similarities = []
    bleu_scores = []
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []

    # Evaluate each prompt
    for i, prompt in enumerate(prompts):
        if not prompt.strip():  # Check if prompt is empty or contains only whitespace
            continue  # Skip empty prompts

        # Generate text
        input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)
        output_ids = model.generate(input_ids,
                                    max_length=500,
                                    num_return_sequences=1,
                                    no_repeat_ngram_size=2,
                                    top_k=top_k,
                                    top_p=top_p,
                                    temperature=temperature,
                                    early_stopping=True,

                                    # pad_token_id = tokenizer.pad_token,
                                    do_sample = True) #to allow varied outputs, negating decoding strategies like beam search / greedy decoding

        generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # Compute metrics
        semantic_score = compute_semantic_similarity(generated_text, reference_lyrics[i])
        bleu_score = compute_bleu_score(generated_text, reference_lyrics[i])
        rouge_scores = compute_rouge_score(generated_text, reference_lyrics[i])

        # Collect scores
        semantic_similarities.append(semantic_score)
        bleu_scores.append(bleu_score)
        rouge1_scores.append(rouge_scores["rouge1"].fmeasure)
        rouge2_scores.append(rouge_scores["rouge2"].fmeasure)
        rougeL_scores.append(rouge_scores["rougeL"].fmeasure)

    # Compute averages
    avg_semantic_similarity = np.mean(semantic_similarities)
    avg_bleu = np.mean(bleu_scores)
    avg_rouge1 = np.mean(rouge1_scores)
    avg_rouge2 = np.mean(rouge2_scores)
    avg_rougeL = np.mean(rougeL_scores)

    print(f"Semantic Similarity Score: {avg_semantic_similarity}")

    # Log results
    results.append({
        "params": {"top_k": top_k, "top_p": top_p, "temperature": temperature},
        "semantic_similarity": avg_semantic_similarity,
        "bleu": avg_bleu,
        "rouge1": avg_rouge1,
        "rouge2": avg_rouge2,
        "rougeL": avg_rougeL
    })

    # Update best parameters
    if avg_semantic_similarity > best_score:
        best_score = avg_semantic_similarity
        best_params = {"top_k": top_k, "top_p": top_p, "temperature": temperature}

# Display best results
print(f"Best Parameters: {best_params}")
print(f"Best Semantic Similarity: {best_score}")

# Display all results
for res in results:
    print(f"Params: {res['params']} -> Semantic: {res['semantic_similarity']}, BLEU: {res['bleu']}, ROUGE-1: {res['rouge1']}, ROUGE-2: {res['rouge2']}, ROUGE-L: {res['rougeL']}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Testing parameters: top_k=10, top_p=0.6, temperature=0.6


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Semantic Similarity Score: 0.3316895763318126
Testing parameters: top_k=10, top_p=0.6, temperature=0.7


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Semantic Similarity Score: 0.31548081453029925
Testing parameters: top_k=30, top_p=0.9, temperature=0.7


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Semantic Similarity Score: 0.3057653971661169
Testing parameters: top_k=30, top_p=0.9, temperature=0.9


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Semantic Similarity Score: 0.28499093673263604
Best Parameters: {'top_k': 20, 'top_p': 0.7, 'temperature': 0.7}
Best Semantic Similarity: 0.3535290025532819
Params: {'top_k': 10, 'top_p': 0.6, 'temperature': 0.6} -> Semantic: 0.3316895763318126, BLEU: 0.013184390557335992, ROUGE-1: 0.2400859955923684, ROUGE-2: 0.06815189436593982, ROUGE-L: 0.15011822251099888
Params: {'top_k': 10, 'top_p': 0.6, 'temperature': 0.7} -> Semantic: 0.34685636082520854, BLEU: 0.01315723292743929, ROUGE-1: 0.2329631613111188, ROUGE-2: 0.06473314478664051, ROUGE-L: 0.14748971127460433
Params: {'top_k': 10, 'top_p': 0.6, 'temperature': 0.9} -> Semantic: 0.3268624463906655, BLEU: 0.015222860710452481, ROUGE-1: 0.2382204417103, ROUGE-2: 0.06363586421727427, ROUGE-L: 0.14691528166410253
Params: {'top_k': 10, 'top_p': 0.7, 'temperature': 0.6} -> Semantic: 0.33142951825776923, BLEU: 0.01629969554060428, ROUGE-1: 0.25624488558215963, ROUGE-2: 0.06583204909616577, ROUGE-L: 0.15160229123795338
Params: {'top_k': 10, 'to

In [38]:
# Prompt for generation
prompt = "I will leave my heart at the door"
input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)

# Generate lyrics - you can change max_length to getlonger songs
output_ids = model.generate(input_ids,
                            max_length=500,
                            num_return_sequences=1,
                            no_repeat_ngram_size=2,
                            top_k = best_params['top_k'],
                            top_p = best_params['top_p'],
                            temperature = best_params['temperature'],
                            early_stopping = True,
                            do_sample = True)

# Decode and print
generated_lyrics = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated_lyrics)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


I will leave my heart at the door and I will walk with you to the end of the world. I have found a way to get to where I am. You will be my friend.

You will not be a burden to me. My heart will never stop.
